<a href="https://colab.research.google.com/github/Geringer13/Netology_pyda/blob/master/Machine_learning_Homework_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Обозначаем корень дерева
1.Находим максимальный Gain (вспоминаем лекцию) для каждого признака.
2.Из полученных Gain’ов признаков выбираем максимальный, и таким образом определяется узел дерева.
3.Далее следуем по левому и правому разбиению, и выполняем пункты 1-3 до тех пор, пока Gain не станет равным 0. В этом случае, мы определяем лист дерева (то есть, возвращаемый ответ)

In [0]:
import pandas as pd
import numpy as np
from collections import Counter

In [0]:
data = pd.read_csv('aac_shelter_outcomes.csv')

In [28]:
data.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,unknown,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,unknown,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,unknown,Rabies Risk,Euthanasia,Unknown


In [0]:
data = data.fillna('unknown')

In [28]:
len(data)

78256

In [0]:
data = data[:100]

In [14]:
Counter(data)

Counter({'age_upon_outcome': 1,
         'animal_id': 1,
         'animal_type': 1,
         'breed': 1,
         'color': 1,
         'date_of_birth': 1,
         'datetime': 1,
         'monthyear': 1,
         'name': 1,
         'outcome_subtype': 1,
         'outcome_type': 1,
         'sex_upon_outcome': 1})

In [0]:
def HEntropy(l):
    length = len(l)
    cnt = Counter(l)
    
    ent = 0
    for cl in cnt.values():
        p = cl / length
        l2 = np.log2(p)
        it = -p * l2
        ent += it
    
    return ent

In [16]:
HEntropy(data)

0.7972627427729667

In [17]:
HEntropy(data['age_upon_outcome'])

3.9040125617909625

In [14]:
HEntropy(data['animal_type'])

1.2590824350392555

In [15]:
HEntropy(data['breed'])

5.757093540044606

In [16]:
HEntropy(data['color'])

5.7374826075977365

In [17]:
HEntropy(data['name'])

9.514150704421482

In [18]:
HEntropy(data['outcome_subtype'])

2.017770867430251

In [19]:
HEntropy(data['outcome_type'])

1.9002931792978204

In [20]:
HEntropy(data['sex_upon_outcome'])

2.0896532063606212

In [0]:
def IG(H, l, i):
    left_l = l[:i]
    right_l = l[i:]
    return H(l) - (len(left_l) / len(l)) * H(left_l) - (len(right_l) / len(l)) * H(right_l)

In [0]:
def test_H(H, l):
    print("{:5} {:3}   {:4} {:4} {:4}".format("#","l","IG","Hl","Hr"))
    print("-"*24)
    for i in range(1,len(l)):
        print("{:2}. {:3}   {:.2f} {:.2f} {:.2f}".format(i, l[i], IG(H, l, i), H(l[:i]), H(l[i:])))

In [29]:
test_H(HEntropy, data['sex_upon_outcome'])

#     l     IG   Hl   Hr  
------------------------
 1. Spayed Female   0.03 0.00 2.11
 2. Neutered Male   0.03 1.00 2.11
 3. Neutered Male   0.02 1.58 2.12
 4. Unknown   0.02 1.50 2.12
 5. Intact Male   0.02 1.92 2.11
 6. Unknown   0.03 1.92 2.10
 7. Spayed Female   0.04 1.95 2.09
 8. Intact Female   0.04 2.00 2.09
 9. Spayed Female   0.02 2.28 2.08
10. Spayed Female   0.02 2.25 2.09
11. Neutered Male   0.02 2.19 2.09
12. Neutered Male   0.01 2.19 2.10
13. Spayed Female   0.01 2.16 2.11
14. Neutered Male   0.01 2.12 2.11
15. Neutered Male   0.01 2.09 2.12
16. Neutered Male   0.01 2.06 2.12
17. Intact Male   0.01 2.01 2.13
18. Spayed Female   0.02 2.06 2.11
19. Spayed Female   0.02 2.04 2.12
20. Unknown   0.02 2.02 2.13
21. Neutered Male   0.02 2.07 2.11
22. Spayed Female   0.02 2.04 2.12
23. Neutered Male   0.02 2.02 2.13
24. Neutered Male   0.02 2.00 2.13
25. Spayed Female   0.02 1.97 2.14
26. Intact Female   0.02 1.96 2.15
27. Spayed Female   0.01 2.04 2.14
28. Intact Female   0.01 

In [30]:
test_H(HEntropy, data['age_upon_outcome'])

#     l     IG   Hl   Hr  
------------------------
 1. 1 year   0.05 0.00 3.89
 2. 1 year   0.06 1.00 3.90
 3. 9 years   0.08 0.92 3.92
 4. 5 months   0.12 1.50 3.87
 5. 4 months   0.14 1.92 3.86
 6. 1 year   0.15 2.25 3.85
 7. 3 years   0.16 2.13 3.87
 8. 1 month   0.15 2.41 3.87
 9. 3 months   0.16 2.64 3.86
10. 1 year   0.15 2.85 3.86
11. 2 years   0.16 2.73 3.87
12. 2 months   0.13 2.92 3.89
13. 4 years   0.11 3.09 3.90
14. 8 years   0.11 3.24 3.89
15. 2 years   0.14 3.37 3.84
16. 1 year   0.13 3.38 3.85
17. 3 weeks   0.14 3.29 3.87
18. 2 months   0.13 3.41 3.85
19. 2 months   0.12 3.43 3.87
20. 8 months   0.12 3.40 3.88
21. 5 months   0.14 3.52 3.83
22. 2 years   0.16 3.53 3.81
23. 2 months   0.15 3.52 3.82
24. 1 year   0.16 3.49 3.83
25. 1 year   0.16 3.43 3.85
26. 4 months   0.17 3.38 3.86
27. 2 months   0.17 3.41 3.86
28. 4 months   0.17 3.38 3.87
29. 2 months   0.19 3.38 3.86
30. 2 weeks   0.19 3.35 3.87
31. 2 months   0.20 3.38 3.84
32. 3 months   0.21 3.35 3.85
33. 3 months

In [37]:
test_H(HEntropy, data['animal_type'])

#     l     IG   Hl   Hr  
------------------------
 1. Dog   0.01 0.00 1.32
 2. Dog   0.00 1.00 1.32
 3. Dog   0.00 0.92 1.33
 4. Other   0.01 0.81 1.33
 5. Dog   0.01 1.37 1.31
 6. Other   0.01 1.25 1.31
 7. Dog   0.02 1.38 1.29
 8. Cat   0.03 1.30 1.29
 9. Cat   0.01 1.44 1.29
10. Cat   0.01 1.49 1.29
11. Dog   0.01 1.49 1.29
12. Dog   0.01 1.46 1.29
13. Dog   0.01 1.42 1.30
14. Dog   0.01 1.38 1.30
15. Dog   0.01 1.34 1.31
16. Dog   0.01 1.30 1.31
17. Cat   0.01 1.26 1.32
18. Dog   0.00 1.30 1.32
19. Cat   0.01 1.27 1.32
20. Cat   0.00 1.30 1.32
21. Cat   0.00 1.31 1.32
22. Dog   0.00 1.32 1.32
23. Cat   0.00 1.30 1.32
24. Cat   0.00 1.31 1.32
25. Dog   0.00 1.31 1.32
26. Dog   0.00 1.30 1.32
27. Dog   0.00 1.28 1.33
28. Dog   0.00 1.26 1.34
29. Cat   0.00 1.25 1.34
30. Cat   0.00 1.26 1.34
31. Cat   0.00 1.26 1.34
32. Dog   0.01 1.26 1.34
33. Cat   0.00 1.25 1.34
34. Dog   0.01 1.25 1.34
35. Cat   0.01 1.24 1.35
36. Dog   0.01 1.24 1.34
37. Dog   0.01 1.24 1.35
38. Cat   0.01 1.23

In [21]:
test_H(HEntropy, data['breed'])

#     l     IG   Hl   Hr  
------------------------
 1. Beagle Mix   0.02 0.00 4.51
 2. Pit Bull   0.06 1.00 4.49
 3. Miniature Schnauzer Mix   0.11 1.58 4.46
 4. Bat Mix   0.14 2.00 4.44
 5. Leonberger Mix   0.16 2.32 4.43
 6. Squirrel Mix   0.21 2.58 4.39
 7. Chihuahua Shorthair Mix   0.24 2.81 4.35
 8. Domestic Shorthair Mix   0.24 3.00 4.36
 9. Domestic Shorthair Mix   0.23 2.95 4.38
10. Domestic Medium Hair Mix   0.23 2.85 4.41
11. Papillon/Border Collie   0.22 3.03 4.41
12. Chihuahua Shorthair/Pomeranian   0.25 3.19 4.37
13. Miniature Schnauzer/Miniature Poodle   0.28 3.33 4.33
14. Labrador Retriever Mix   0.31 3.47 4.29
15. Rat Terrier Mix   0.30 3.59 4.30
16. Pit Bull Mix   0.30 3.70 4.28
17. Domestic Shorthair Mix   0.28 3.81 4.28
18. German Shepherd/Labrador Retriever   0.28 3.73 4.31
19. Domestic Shorthair Mix   0.30 3.83 4.27
20. Domestic Shorthair Mix   0.30 3.74 4.30
21. Domestic Shorthair Mix   0.30 3.65 4.33
22. Beagle Mix   0.30 3.57 4.36
23. Domestic Shorthair Mix   0

In [22]:
test_H(HEntropy, data['color'])

#     l     IG   Hl   Hr  
------------------------
 1. White/Brown   0.05 0.00 4.96
 2. Blue/White   0.08 1.00 4.96
 3. White   0.11 1.58 4.95
 4. Brown   0.12 2.00 4.95
 5. Brown/White   0.13 2.32 4.96
 6. Tan   0.15 2.58 4.95
 7. Brown   0.16 2.81 4.95
 8. Blue Tabby/White   0.18 2.75 4.96
 9. White/Black   0.19 2.95 4.95
10. Black/White   0.20 3.12 4.94
11. Black/White   0.18 3.28 4.97
12. Black   0.18 3.25 4.99
13. White   0.16 3.39 5.00
14. Black   0.18 3.38 5.00
15. White/Black   0.19 3.37 5.02
16. Brown Brindle/White   0.21 3.38 5.01
17. Black/White   0.23 3.50 4.97
18. Black/White   0.24 3.46 5.00
19. Brown Tabby   0.25 3.41 5.02
20. Brown Tabby/White   0.22 3.52 5.04
21. Black Tabby   0.22 3.63 5.03
22. Tan/White   0.24 3.73 4.99
23. Orange Tabby   0.23 3.83 4.99
24. Black/White   0.24 3.83 4.99
25. Brown Brindle   0.25 3.78 5.02
26. Brown Brindle/Black   0.27 3.87 4.98
27. Yellow   0.28 3.95 4.94
28. Blue/Tan   0.28 4.04 4.93
29. Seal Point   0.29 4.11 4.90
30. Black   0.30 

In [31]:
test_H(HEntropy, data['outcome_type'])

#     l     IG   Hl   Hr  
------------------------
 1. Transfer   0.01 0.00 1.85
 2. Adoption   0.03 0.00 1.85
 3. Transfer   0.02 0.92 1.85
 4. Euthanasia   0.02 0.81 1.86
 5. Transfer   0.02 1.37 1.85
 6. Euthanasia   0.03 1.25 1.85
 7. Transfer   0.04 1.38 1.84
 8. Adoption   0.04 1.30 1.84
 9. Adoption   0.03 1.44 1.85
10. Return to Owner   0.02 1.49 1.85
11. Transfer   0.01 1.79 1.84
12. Adoption   0.01 1.73 1.84
13. Return to Owner   0.01 1.74 1.85
14. Return to Owner   0.01 1.84 1.83
15. Transfer   0.02 1.89 1.81
16. Transfer   0.02 1.85 1.82
17. Transfer   0.02 1.81 1.82
18. Adoption   0.03 1.77 1.82
19. Adoption   0.02 1.78 1.83
20. Transfer   0.02 1.78 1.84
21. Adoption   0.02 1.75 1.84
22. Adoption   0.02 1.75 1.85
23. Adoption   0.01 1.74 1.86
24. Transfer   0.01 1.73 1.86
25. Adoption   0.01 1.71 1.87
26. Transfer   0.01 1.70 1.87
27. Adoption   0.02 1.68 1.88
28. Transfer   0.02 1.67 1.89
29. Transfer   0.02 1.65 1.89
30. Transfer   0.02 1.64 1.90
31. Adoption   0.03 1.6

In [24]:
data.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


In [0]:
new = data['outcome_type']

In [34]:
new.unique()

array(['Transfer', 'Adoption', 'Euthanasia', 'Return to Owner', 'Died'],
      dtype=object)

In [0]:
new.replace(['Transfer', 'Adoption', 'Euthanasia', 'Return to Owner', 'Died'], [1, 2, 3, 4, 5], inplace=True)

In [50]:
new

0     1
1     1
2     2
3     1
4     3
     ..
95    3
96    1
97    1
98    3
99    2
Name: outcome_type, Length: 100, dtype: int64

In [51]:
test_H(HEntropy, new)

#     l     IG   Hl   Hr  
------------------------
 1.   1   0.01 0.00 1.85
 2.   2   0.03 0.00 1.85
 3.   1   0.02 0.92 1.85
 4.   3   0.02 0.81 1.86
 5.   1   0.02 1.37 1.85
 6.   3   0.03 1.25 1.85
 7.   1   0.04 1.38 1.84
 8.   2   0.04 1.30 1.84
 9.   2   0.03 1.44 1.85
10.   4   0.02 1.49 1.85
11.   1   0.01 1.79 1.84
12.   2   0.01 1.73 1.84
13.   4   0.01 1.74 1.85
14.   4   0.01 1.84 1.83
15.   1   0.02 1.89 1.81
16.   1   0.02 1.85 1.82
17.   1   0.02 1.81 1.82
18.   2   0.03 1.77 1.82
19.   2   0.02 1.78 1.83
20.   1   0.02 1.78 1.84
21.   2   0.02 1.75 1.84
22.   2   0.02 1.75 1.85
23.   2   0.01 1.74 1.86
24.   1   0.01 1.73 1.86
25.   2   0.01 1.71 1.87
26.   1   0.01 1.70 1.87
27.   2   0.02 1.68 1.88
28.   1   0.02 1.67 1.89
29.   1   0.02 1.65 1.89
30.   1   0.02 1.64 1.90
31.   2   0.03 1.62 1.90
32.   2   0.03 1.61 1.91
33.   2   0.03 1.61 1.92
34.   2   0.03 1.60 1.92
35.   1   0.03 1.59 1.93
36.   2   0.03 1.58 1.94
37.   2   0.04 1.57 1.95
38.   5   0.04 1.56 1.9